# PyHEP 2022 - Using C++ From Numba, Fast and Automatic

## Numba

The scientific community has developed various techniques to write codes easily while also accelerating their execution speeds. Numba is one such library. Numba translates Python functions into optimized machine code at runtime using the industry-standard LLVM compiler framework. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN.

### Numba: Tradeoff between flexibility and performance

Python has the flexibility of converting easily between different data types. This is because each python object is a PyObject that can store any datatype that is used in Python.

```python
f = 0.5
```

![Boxing in Python](box_pyobject.svg "Boxing float in PyObject")

So when you store a floating point number in a variable in python. Python first converts it to a Pyobject, which is called boxing, and then the pointer to this Pyobject is what the variable stores. Whenever the native value, the floating point number in this case, is required for any calculations it needs to be unboxed from the PyObject and then used for calculations.

```python
f = f ** 2
```

![Unboxing in Python](unbox_pyobject.svg "Unboxing float from PyObject")

These boxing and unboxing operations are deterimental to performance but provide the necessary flexibility for Python duck typing.

Numba on the other hand gets rid of this flexibility for performance. It unboxes the inputs of the function and the whole function is run on native values and not PyObjects. At the end the output is boxed so that Python can use it. For this to work numba needs to figure out the types of not only the input and output but the intermediate variables as well.

![Numba working](numba.svg "Numba only deals with native values in the LLVM IR")

The drawback to this approach is that if the types in the program that are not determinable the speed up will be minimal.

Once the types are inferred Numba converts the Python code into LLVM IR. LLVM IR is a representation that is created by LLVM and used in it's compiler framework as a device independent representation which can be easily converted into assembly using LLVM tools.

```c++
define linkonce_odr dso_local void @_ZN1CC2Ei(ptr noundef nonnull align 4 dereferenceable(4) %this, i32 noundef %x) unnamed_addr #2 comdat align 2 !dbg !34 {
entry:
  %this.addr = alloca ptr, align 8
  %x.addr = alloca i32, align 4
  store ptr %this, ptr %this.addr, align 8
  call void @llvm.dbg.declare(metadata ptr %this.addr, metadata !35, metadata !DIExpression()), !dbg !37
  store i32 %x, ptr %x.addr, align 4
  call void @llvm.dbg.declare(metadata ptr %x.addr, metadata !38, metadata !DIExpression()), !dbg !39
  %this1 = load ptr, ptr %this.addr, align 8
  %i = getelementptr inbounds %class.C, ptr %this1, i32 0, i32 0, !dbg !40
  %0 = load i32, ptr %x.addr, align 4, !dbg !41
  store i32 %0, ptr %i, align 4, !dbg !40
  ret void, !dbg !42
}
```

The snippet above is in LLVM IR. If you have seen assembly code before you must think it looks pretty similar. Since LLVM doesn't have a python package to build and execute the IR, numba has created llvmlite that allows it to do the same from python itself.

### Performance benefits from Numba

To measure the performance benefits from numba we use a simple function. This function calculates the `tanh` of the trace of a numpy matrix and adds it back to the whole matrix. __The only difference between the two is the Numba decorator on Line 15.__ This decorator instructs numba to convert the function into LLVM IR.

In [1]:
from numba import jit
import numpy as np
import time

################ Pure Python ###############
# Function is not compiled and runs in byte code
def python_trace(a):
    trace = 0.0
    for i in range(a.shape[0]):
        trace += np.tanh(a[i, i])
    return a + trace

################ Numba ###############
# Function is compiled and runs in machine code
@jit(nopython=True) # <--------------- Numba decorator
def numba_trace(a):
    trace = 0.0
    for i in range(a.shape[0]):
        trace += np.tanh(a[i, i])
    return a + trace

In [2]:
def measure_execution(func, args):
    start = time.perf_counter()
    results = func(*args)
    end = time.perf_counter()
    elapsed = end - start
    return elapsed, results

x = np.arange(100).reshape(10, 10)

numba_warmup, _ = measure_execution(numba_trace, (x,))
python_elapsed, _ = measure_execution(python_trace, (x,))
numba_elapsed, _ = measure_execution(numba_trace, (x,))

In [3]:
print(f"Numba Function : Warmup  = {numba_warmup:.7f}s")
print(f"Python Function: Elapsed = {python_elapsed:.7f}s")
print(f"Numba Function : Elapsed = {numba_elapsed:.7f}s")

Numba Function : Warmup  = 0.3749311s
Python Function: Elapsed = 0.0000542s
Numba Function : Elapsed = 0.0000053s


These results show that in the first run numba takes a lot more time than a conventional python function. This is because it takes time to load necessary modules and compile the function into LLVM IR whereas the conventional python function execution does have to go through these extra steps. After the compilation is done numba can provide a speedup of one or two orders of magnitude depending on the program itself. Thus numba is used when the same function has to be run a lot of times.

## Cppyy

Cppyy is an automatic, run-time, Python-C++ bindings generator, for calling C++ from Python and Python from C++. It is the base for PyROOT which is the Python frontend for ROOT, which is the data analysis framework used for Physics analysis. ROOT has a C++ interpreter Cling built into it which also provides Cppyy the necessary information for it to use C++ features.

Cppyy as PyROOT has been employed by many experiments to use their own C++ data models in Python. 

### Cppyy: Why should you care about C++?

__Most of the codebase used by scientific community is in C++.__ Converting these to a language such as Python will require a huge workforce and would also suffer performance hits thus it would not be a feasible option. Using cppyy, the C++ data models, that the respective HEP experiments have, are readily available to the user while also providing the ease of protoptyping that Python allows.

PyROOT has been a major boon for people who want to use Python. But previously numba did not support PyROOT objects as it did not understand them. Due to the cppyy extension numba can now interact with PyROOT and thus __anyone using PyROOT can now use numba to their advantage by using `cppyy.numba_ext`.__

### Benefits of using Cppyy with Numba?

1) __Numba makes loops fast:__ When using cppyy (PyROOT) with python, the loops in python are slower as compared to languages such as C/C++. Numba alleviates this problem and can make it as fast as C without much code instrumentation.

2) __Code completely in python:__ This makes debugging easier. To debug numba instrumented code you can either comment out the instrumentation line and debug the code as you would do in Python or use gdb using `numba.gdb`. Numba also has a variety of flags that can be turned on to see tracebacks and the intermediate steps taken by numba. _This is easier than to debug a code that is setup in Python and uses RDF for hotspots._

3) __No conversions in the IR:__ Cppyy can be converted to LLVM IR cleanly, that is no boxing and unboxing is required, so we do not spend any time in type conversions and gain the maximum amount of speedup possible.

4) __Two worlds closer together:__ You can switch easily between C++ and Python as and when you want.

### Performance

Similar to the tanh example used to compare Numba vs Python we use the `std::tanh` from C++ to compare the performance against numba. We just replace the `np.tanh` function and no extra changes are done.

In [4]:
import cppyy
import cppyy.numba_ext

################ Cppyy ###############
# Function is compiled and runs in machine code
@jit(nopython=True)
def cppyy_numba_trace(a):
    trace = 0.0
    for i in range(a.shape[0]):
        trace += cppyy.gbl.tanh(a[i, i]) # <---------------- Replaces np.tanh
    return a + trace

In [5]:
cppyy_warmup, _ = measure_execution(cppyy_numba_trace, (x,))
cppyy_elapsed, _ = measure_execution(cppyy_numba_trace, (x,))

print(f"Numba Function : Elapsed (with comp)  = {numba_warmup:.7f}s")
print(f"Cppyy Function : Elapsed (with comp)  = {cppyy_warmup:.7f}s")
print()
print(f"Python Function: Elapsed (2nd run)    = {python_elapsed:.7f}s")
print(f"Numba Function : Elapsed (after comp) = {numba_elapsed:.7f}s")
print(f"Cppyy Function : Elapsed (after comp) = {cppyy_elapsed:.7f}s")

Numba Function : Elapsed (with comp)  = 0.3749311s
Cppyy Function : Elapsed (with comp)  = 0.1676025s

Python Function: Elapsed (2nd run)    = 0.0000542s
Numba Function : Elapsed (after comp) = 0.0000053s
Cppyy Function : Elapsed (after comp) = 0.0000072s


The result show that overhead for using cppyy in a numba function is minimal as the time elapsed is almost similar to the numba only function.

## Features

### 1) Plug and Play

To use the extension you just need to import `cppyy.numba_ext` and then you can use C++ functions in numba directly.

In the example shown below `sqrt` is a C++ function that can be used direcly inside the numba jitted function with the help of the extension.

In [6]:
import numba
import cppyy
import cppyy.numba_ext   # <------- Imports the necessary information for numba to work with cppyy
import math

@numba.jit(nopython=True)
def cpp_sqrt(x):
    return cppyy.gbl.sqrt(x) # <------------ Direct use, no extra setup required

print("Sqrt of 4: ", cpp_sqrt(4.0))
print("Sqrt of Pi: ", cpp_sqrt(math.pi))

Sqrt of 4:  2.0
Sqrt of Pi:  1.7724538509055159


### 3) Template instantiation

Cppyy supports template instantiation which gives you access to an important feature set in C++ that is used abundently in lot of places. This extension extends that support to numba too so any templated C++ function can be used in numba. Below we have a templated square function and depending on the type of the matrix the extension instantiates the required template argument.

In [7]:
import cppyy
import cppyy.numba_ext
import numba
import numpy as np

cppyy.cppdef("""
template<typename T>
T square(T t) { return t*t; }
""")

@numba.jit(nopython=True)
def tsa(a):
    total = type(a[0])(0)
    for i in range(len(a)):
        total += cppyy.gbl.square(a[i])
    return total

a = np.array(range(10), dtype=np.float32)
print("Float array: ", a)
print("Sum of squares: ", tsa(a))
print()
a = np.array(range(10), dtype=np.int32)
print("Integer array: ", a)
print("Sum of squares: ", tsa(a))

Float array:  [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
Sum of squares:  285.0

Integer array:  [0 1 2 3 4 5 6 7 8 9]
Sum of squares:  285


### 2) Overload selection

In [8]:
cppyy.cppdef("""
int mul(int x) { return x * 2; }
float mul(float x) { return x * 3; }
""")

@numba.jit(nopython=True)
def oversel(a):
    total = type(a[0])(0)
    for i in range(len(a)):
        total += cppyy.gbl.mul(a[i])
    return total

a = np.array(range(10), dtype=np.float32)
print("Array: ", a)
print("Overload selection output: ", oversel(a))

a = np.array(range(10), dtype=np.int32)
print("Array: ", a)
print("Overload selection output: ", oversel(a))

Array:  [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
Overload selection output:  135.0
Array:  [0 1 2 3 4 5 6 7 8 9]
Overload selection output:  90


## Demos

### 1) Numba physics example

Taken from:
https://github.com/numba/numba-examples/blob/master/examples/physics/lennard_jones/numba_scalar_impl.py

$$V_{LJ}(r) = 4\varepsilon[(\frac{\sigma}{r})^{12} - (\frac{\sigma}{r})^6]$$

$$ \varepsilon = 1, \sigma = 1$$

In [9]:
import numba
import cppyy
import cppyy.numba_ext

cppyy.cppdef("""
#include <vector>

struct Atom {
    float x;
    float y;
    float z;
};

std::vector<Atom> atoms = {{1, 2, 3}, {2, 3, 4}, {3, 4, 5}, {4, 5, 6}, {5, 6, 7}};
""")

@numba.njit
def lj_numba_scalar(r):
    sr6 = (1./r)**6
    pot = 4.*(sr6*sr6 - sr6)
    return pot


@numba.njit
def distance_numba_scalar(atom1, atom2):
    dx = atom2.x - atom1.x
    dy = atom2.y - atom1.y
    dz = atom2.z - atom1.z

    r = (dx * dx + dy * dy + dz * dz) ** 0.5

    return r



def potential_numba_scalar(cluster):
    energy = 0.0
    for i in range(cluster.size() - 1):
        for j in range(i + 1, cluster.size()):
            r = distance_numba_scalar(cluster[i], cluster[j])
            e = lj_numba_scalar(r)
            energy += e
            
    return energy

print("Total lennard jones potential =", potential_numba_scalar(cppyy.gbl.atoms))

Total lennard jones potential = -0.5780277345740283


### 2) Using the extension with PyROOT

To use the extension with PyROOT we just need to import `cppyy.numba_ext`.

In the example we use the TLorentzVector class from ROOT. TLorentzVector is a root class with four properties:
`Px`, `Py`, `Pz` and `E`

It also provides the transverse momentum `Pt` to the user which can be calculated by:

$$Pt = \sqrt{Px^2+Py^2}$$

In [10]:
########################################## Setup Code ###############################
import numba
import math
import ROOT
import cppyy.numba_ext
import time

ROOT.gInterpreter.Declare("""
std::vector<TLorentzVector> vec_lv;

const int no_of_samples = 1000;

void fill() {
  vec_lv.reserve(no_of_samples);
  TRandom3 R(111);
  
  for (int i = 0; i < no_of_samples; ++i) {
    double Px = R.Gaus(0,10);
    double Py = R.Gaus(0,10);
    double Pz = R.Gaus(0,10);
    double E  = R.Gaus(100,10);
    vec_lv.push_back(TLorentzVector(Px, Py, Pz, E));
  }
}
""")
ROOT.gInterpreter.ProcessLine("""
fill();
""")

print()

Welcome to JupyROOT 6.27/01



In this example we calculate the same using Python and show how we can speed up the calculation using numba.
The `calc_pt` function uses pure python to calculate `Pt` whereas the `numba_calc_pt` uses numba to do the same. As before the only __difference between the two is the numba decorator__ so you do not need to change anything.

In [11]:
def calc_pt(lv):
    return math.sqrt(lv.Px() ** 2 + lv.Py() ** 2)

def calc_pt_vec(vec_lv):
    pt = []
    for i in range(vec_lv.size()):
        pt.append((calc_pt(vec_lv[i]), vec_lv[i].Pt()))
    return pt


@numba.njit
def numba_calc_pt(lv):
    return math.sqrt(lv.Px() ** 2 + lv.Py() ** 2)

def numba_calc_pt_vec(vec_lv):
    pt = []
    for i in range(vec_lv.size()):
        pt.append((numba_calc_pt(vec_lv[i]), vec_lv[i].Pt()))
    return pt


In [12]:
numba_warmup, _ = measure_execution(numba_calc_pt, (ROOT.vec_lv[0], ))
python_elapsed, _ = measure_execution(calc_pt_vec, (ROOT.vec_lv, ))
numba_elapsed, pt = measure_execution(numba_calc_pt_vec, (ROOT.vec_lv, ))

print(f"Numba'd    : Warmup  = {numba_warmup  :.5f}s")
print(f"Pure Python: Elapsed = {python_elapsed:.5f}s")
print(f"Numba'd    : Elapsed = {numba_elapsed :.5f}s")

print(f"Speedup              = {python_elapsed / numba_elapsed:.5f}x")

no_of_samples = 3
print("\nCalc pT \tActual pT")
print("---------------------------")
print(*(f"{x:2.5f} \t{y:2.5f}" for x,y in pt[:no_of_samples]), sep="\n")

if False in tuple(x==y for x, y in pt):
    print("\nSome values do not match")
else:
    print("\nAll values for pT match")

Numba'd    : Warmup  = 0.04911s
Pure Python: Elapsed = 0.00992s
Numba'd    : Elapsed = 0.00409s
Speedup              = 2.42418x

Calc pT 	Actual pT
---------------------------
8.95222 	8.95222
4.11973 	4.11973
25.97929 	25.97929

All values for pT match


### 3) RDF

You can also use it inside RDF through `ROOT.Numba.Declare`. Underneath is a simple example where it is used to calculate the power function.

In [13]:
import numba
import ROOT
import cppyy.numba_ext

ROOT.gInterpreter.Declare("""
double cpppow(double x, int y) { return pow(x, y); }
""")

@ROOT.Numba.Declare(['double', 'int'], 'double')
def pypownd(x, y):
    return ROOT.cpppow(x, y) # <--------- Numba.Declare supports ROOT python due to the numba extension


ROOT.gInterpreter.ProcessLine("""
cout << "2^3 = " << Numba::pypownd(2, 3) << endl
     << "4^5 = " << Numba::pypownd(4, 5) << endl;""")
print()

# Or we can use the callable as well within a RDataFrame workflow.
data = ROOT.RDataFrame(4).Define('x', '(float)rdfentry_')\
                         .Define('x_pow3', 'Numba::pypownd(x, 3)')\
                         .AsNumpy()
 
print('pypownd({}, 3) = {}'.format(data['x'], data['x_pow3']))


pypownd([0. 1. 2. 3.], 3) = [ 0.  1.  8. 27.]
2^3 = 8
4^5 = 1024


## Future work

1) __Complete C++ feature support__:
  - Returning C++ objects from functions
  - Memory management
  - Constructor support
  - Virtual inheritance

2) __Inlining__:

For the code:
```python
def numba_calc_pt(lv):
    return math.sqrt(lv.Px() ** 2 + lv.Py() ** 2)
```

The equivalent LLVM IR is:
```c++
define i32 @_ZN8__main__13numba_calc_ptB2v1B38c8tJTIcFHzwl2ILiXkcBV0KBSgP9CGZpAgA_3dE28CppClass_28TLorentzVector_29(double* noalias nocapture %retptr, { i8*, i32, i8* }** noalias nocapture readnone %excinfo, { i32, i32, i32, i32, i32, i32 }* %arg.lv) local_unnamed_addr {
entry:
  %.5 = bitcast { i32, i32, i32, i32, i32, i32 }* %arg.lv to i8*
  %.614 = load double (i8*)*, double (i8*)** bitcast (i8** @numba.dynamic.globals.7fbfd71f7080 to double (i8*)**), align 8
  %.8 = tail call double %.614(i8* %.5)
  %.147.i = fmul double %.8, %.8
  %.3615 = load double (i8*)*, double (i8*)** bitcast (i8** @numba.dynamic.globals.7fbfd6d7b080 to double (i8*)**), align 8
  %.38 = tail call double %.3615(i8* %.5)
  %.147.i9 = fmul double %.38, %.38
  %.63 = fadd double %.147.i, %.147.i9
  store double %.63, double* %retptr, align 8
  ret i32 0
}
```
For best performance the line ```%.3615 = load double (i8*)*, double (i8*)** bitcast (i8** @numba.dynamic.globals.7fbfd6d7b080 to double (i8*)**), align 8``` should be replaced by memory access and that will require rebasing llvmlite on top of Cling. This will allow symbols of C++ that Cling parsed are available at LLVM IR level and so we can replace the function call easily.

3) __GPU support__

4) __Automatic parallelization__: We want to support automatic parallelization using OpenMP


# Conclusion

This extension:

#### 1) __Is easy to use__: Just import `cppyy.numba_ext`.

#### 2) __Enables you to use PyROOT with Numba.__

#### 3) __Makes code debugging easy by using a full Python__

$$$$

----------------------------------------
# $$Thankyou$$
----------------------------------------

$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$
$$$$

$$Extra Examples$$

In [ ]:
import numba
import cppyy
import numpy as np

cppyy.cppdef("""
float arr1[] = {0.1, 0.2, 0.3, 0.4, 0.5};
""")

@numba.njit
def square_arr(arr):
    ret = []
    for i in range(5):
        x = arr[i] ** 2
        ret.append(x)
            
    return np.array(ret)

print(square_arr(cppyy.gbl.arr1))

In [ ]:
import cppyy
import numba
import numpy as np

cppyy.cppdef("""\
class MyData {
public:
    MyData(int i, int j) : fField1(i), fField2(j) {}

public:
    int get_field1() { return fField1; }
    int get_field2() { return fField2; }

    MyData copy() { return *this; }

public:
    int fField1;
    int fField2;
};""")

@numba.jit(nopython=True)
def tsdf(a, d):
    total = type(a[0])(0)
    for i in range(len(a)):
        total += a[i] + d.fField1 + d.fField2
    return total

d = cppyy.gbl.MyData(5, 6)
a = np.array(range(10), dtype=np.int32)
print(tsdf(a, d))

# example of method calls
@numba.jit(nopython=True)
def tsdm(a, d):
    total = type(a[0])(0)
    for i in range(len(a)):
        total += a[i] +  d.get_field1() + d.get_field2()
    return total

print(tsdm(a, d))

# example of object return by-value
@numba.jit(nopython=True)
def tsdcm(a, d):
    total = type(a[0])(0)
    for i in range(len(a)):
        total += a[i] + d.copy().fField1 + d.get_field2()
    return total

print(tsdcm(a, d))



In [ ]:
import numba
import cppyy

cppyy.cppdef("""
float arr2[] = {0.1, 0.2, 0.3, 0.4, 0.5};
""")

@numba.njit
def sum_arr(arr):
    energy = 0.0
    for i in range(5):
        energy += arr[i]
            
    return energy

print(sum_arr(cppyy.gbl.arr2))